In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm # a nice progress bar

In [ ]:
# generate a dataset of 1D data from a mixture of two Gaussians
# this is a simple example, but you can use any distribution
data_distribution = torch.distributions.mixture_same_family.MixtureSameFamily(
    torch.distributions.Categorical(torch.tensor([1, 2])),  #ratio 1:2 of falling into one distribution vs the other
    torch.distributions.Normal(torch.tensor([-4., 4.]), torch.tensor([1., 1.]))
)

In [ ]:
dataset = data_distribution.sample(torch.Size([10000]))  # create training data set
dataset_validation = data_distribution.sample(torch.Size([1000])) # create validation data set

# Plot the true distribution
sns.histplot(dataset.numpy(), stat='density', bins=80, label='True Data')
plt.legend()
plt.title("True Data Distribution")
plt.show()

In [ ]:
#Set hyperparameters
TIME_STEPS = 250
BETA = torch.tensor(0.02)
ALPHA = 1.0 - BETA
ALPHABAR = torch.cumprod(torch.full((TIME_STEPS,), ALPHA), dim=0)

N_EPOCHS = 1000
BATCH_SIZE = 64
LEARNING_RATE = 8e-4


In [ ]:
class NoisePredictorMLP(nn.Module):
    """
    inputs: a noisy sample (xt), time step (t)
    outputs: a single scalar per sample -- the predicted standard normal noise
    """
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x, t):
        # Normalize time and concatenate with x_t
        t = t.float().unsqueeze(1) / TIME_STEPS
        x = x.unsqueeze(1)
        return self.model(torch.cat([x, t], dim=1)).squeeze(1)

g = NoisePredictorMLP()
optimizer = torch.optim.Adam(g.parameters(), lr=LEARNING_RATE)
loss_fn = nn.MSELoss()


In [ ]:
#training loop
epochs = tqdm(range(N_EPOCHS))
for epoch in epochs:
    g.train()
    indices = torch.randperm(len(dataset))
    for i in range(0, len(dataset) - BATCH_SIZE, BATCH_SIZE): #skip final incomplete batch
        x0 = dataset[indices[i:i + BATCH_SIZE]]

        # Sample time step and noise
        t = torch.randint(0, TIME_STEPS, (BATCH_SIZE,)) #generate random time steps in the correct shape (BATCH_SIZE,)
        alpha_bar = ALPHABAR[t].unsqueeze(1) #generate alphabar corresponding to each time step
        epsilon = torch.randn_like(x0)

        # Create noisy version x_t
        xt = torch.sqrt(alpha_bar) * x0.unsqueeze(1) + torch.sqrt(1 - alpha_bar) * epsilon.unsqueeze(1)
        xt = xt.squeeze(1)

        # Predict epsilon and compute loss
        pred_epsilon = g(xt, t)
        loss = loss_fn(pred_epsilon, epsilon)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [ ]:
@torch.no_grad()
def sample_reverse(g, count):
    """
    Sample from the model by applying the reverse diffusion process

    ----------
    Parameters
    ----------
    g : torch.nn.Module
        The neural network that predicts the noise added to the data
    count : int
        The number of samples to generate in parallel

    ----------
    Returns
    ----------
    x : torch.Tensor
        The final sample from the model
    """
    g.eval()
    x = torch.randn(count)  # Start from pure noise

    for t in reversed(range(1, TIME_STEPS)):
        t_batch = torch.full((count,), t, dtype=torch.long) #feed the current timestep t into the network for each sample in the batch
        beta = BETA
        alpha = ALPHA
        alpha_bar = ALPHABAR[t]

        z = torch.randn_like(x) if t > 1 else torch.zeros_like(x) #generate z to add stochasticity to the reverse step
        pred_epsilon = g(x, t_batch) #predict the standard normal noise that was added to get xt using trained NN

        x = (1 / torch.sqrt(alpha)) * (
            x - (1 - alpha) / torch.sqrt(1 - alpha_bar) * pred_epsilon
        ) + torch.sqrt(beta) * z

    return x #final output x0


In [ ]:
# ---------------------------------------------
#Plot generated samples against the underlying function used to generate training data
# ---------------------------------------------
samples = sample_reverse(g, 1000).numpy()

sns.histplot(samples, stat='density', bins=50, color='red', label='Model Samples')
sns.kdeplot(dataset.numpy(), color='blue', label='True Data', linewidth=2)
plt.title("Comparison: Generated vs True Distribution")
plt.legend()
plt.show()


In [ ]:
# ---------------------------------------------
# Visualization: How samples are transformed with increasing diffusion steps
# ---------------------------------------------

# Sample from the actual data distribution (mixture of Gaussians)
x0_vis = data_distribution.sample(torch.Size([200]))  # same as training distribution

# Select timesteps to visualize
timesteps = [0, 50, 100, 150, 200, 249]
xt_distributions = []

# Apply forward diffusion at selected timesteps
for t in timesteps:
    alpha_bar_t = ALPHABAR[t]
    eps = torch.randn_like(x0_vis)
    xt = torch.sqrt(alpha_bar_t) * x0_vis + torch.sqrt(1 - alpha_bar_t) * eps
    xt_distributions.append(xt)

# Plot the evolution of the distribution over time
fig, axes = plt.subplots(2, 3, figsize=(15, 8), sharey=True)
axes = axes.flatten()
for i, t in enumerate(timesteps):
    sns.histplot(xt_distributions[i].numpy(), bins=50, stat='density', ax=axes[i], color='gray')
    axes[i].set_title(f"Timestep t = {t}")
    axes[i].set_xlim([-10, 10])
    axes[i].set_xlabel("x_t")
    axes[i].set_ylabel("Density")

plt.suptitle("Forward Diffusion: How Clean Samples Become Noised Over Time", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:

# ---------------------------------------------
# Plot reverse diffusion trajectories (it is evident that we wind up with a multimodal mixed model distribution -- with more trajectories winding up at one mode (~2:1 ratio))
# ---------------------------------------------
def trace_multiple_trajectories(g, time_steps, beta, alpha, alpha_bar, num_samples=20):
    """
    Tracks how multiple samples evolve from noise to data over time.
    Returns a list of trajectories, one per sample.
    """
    g.eval()
    x = torch.randn(num_samples)  # Start from x_T ~ N(0, 1)
    trajectories = [ [x[i].item()] for i in range(num_samples) ]

    for t in reversed(range(1, time_steps)):
        t_tensor = torch.full((num_samples,), t, dtype=torch.long)
        z = torch.randn_like(x) if t > 1 else torch.zeros_like(x)
        pred_epsilon = g(x, t_tensor)

        x = (1 / torch.sqrt(alpha)) * (
            x - (1 - alpha) / torch.sqrt(1 - alpha_bar[t]) * pred_epsilon
        ) + torch.sqrt(beta) * z

        for i in range(num_samples):
            trajectories[i].append(x[i].item())

    return [traj[::-1] for traj in trajectories]  # Now traj[0] is x_0, traj[-1] is x_T

# Get the trajectories
trajectories = trace_multiple_trajectories(g, TIME_STEPS, BETA, ALPHA, ALPHABAR)

# Plot them from right (x_T) to left (x_0)
plt.figure(figsize=(14, 6))
for i, traj in enumerate(trajectories):
    plt.plot(range(TIME_STEPS), traj, label=f"Sample {i+1}")
plt.gca().invert_xaxis()  # Flip x-axis: right = noise, left = structure
plt.title("Denoising Trajectories (Right = Noise, Left = Generated Sample)")
plt.xlabel("Timestep t (→ backward in time)")
plt.ylabel("x_t value")
plt.grid(True)
plt.legend(loc="upper right", fontsize="small", ncol=2)
plt.tight_layout()
plt.show()


In [ ]:
import json
import urllib.request
import ipykernel
from notebook import notebookapp

# Get current kernel connection file
connection_file = ipykernel.get_connection_file()
kernel_id = connection_file.split('-')[-1].split('.')[0]

# Find all running servers
for srv in notebookapp.list_running_servers():
    try:
        url = srv['url'] + 'api/sessions'
        token = srv.get('token', '')
        if token:
            url += '?token=' + token
        sessions = json.load(urllib.request.urlopen(url))
        for sess in sessions:
            if sess['kernel']['id'] == kernel_id:
                notebook_path = sess['notebook']['path']
                print("✅ Current notebook path:", notebook_path)
    except Exception as e:
        print("Error:", e)
